In [17]:
import pandas as pd

# Define la ruta del archivo Excel
# ruta_archivo = 'C:\\Users\\lbarc\\OneDrive - Universidad Pontificia Comillas\\ICAI 11\\Machine Learning III\\Datos Dia 1\\STRIKE_REPORTS.xlsx'

# Carga el archivo Excel en un DataFrame de pandas
# df = pd.read_excel(ruta_archivo, sheet_name='STRIKE_REPORTS')


In [18]:
# Elimina filas donde cualquiera de las columnas especificadas tenga un valor NaN
# df = df.dropna(subset=['HEIGHT', 'SPEED', 'DISTANCE', 'NUM_STRUCK'])


In [12]:
# Suma las columnas y guarda el resultado en una nueva columna
# df['COST_INFL_ADJ'] = df['COST_REPAIRS_INFL_ADJ'] + df['COST_OTHER_INFL_ADJ']


In [14]:
# Nombre del archivo de salida
# nombre_archivo_salida = 'datos_limpios.xlsx'

# Exporta el DataFrame limpio a un archivo Excel
# df.to_excel(nombre_archivo_salida, index=False)

In [16]:
import pandas as pd

# Define la ruta del archivo Excel
ruta_archivo = 'datos_limpios.xlsx'

# Carga el archivo Excel en un DataFrame de pandas
df = pd.read_excel(ruta_archivo)

df.head()

,TIME_OF_DAY,AIRCRAFT,AC_MASS,PHASE_OF_FLIGHT,HEIGHT,SPEED,DISTANCE,SPECIES,NUM_STRUCK,COST_INFL_ADJ
0,Day,B-727-100,4,Take-off Run,0,100,0.0,Unknown bird - medium,1,NaN
1,Day,DC-9,4,Landing Roll,0,100,0.0,Unknown bird - small,2-10,NaN
2,Day,MD-11,4,Landing Roll,0,120,0.0,Unknown bird - small,1,NaN
3,Day,MD-80,4,Landing Roll,0,125,0.0,American kestrel,1,NaN
4,Day,B-737,4,Landing Roll,0,100,0.0,Gulls,1,NaN


In [20]:
# Análisis de la distribución de categorías
print(df['TIME_OF_DAY'].value_counts())
print(df['AIRCRAFT'].value_counts())
print(df['AC_MASS'].value_counts())

Day      33810
Night    14540
Dusk      2593
Dawn      2026
Name: TIME_OF_DAY, dtype: int64
B-737-700          5461
B-737-800          3242
B-737-300          2695
A-320              2308
CRJ100/200         2009
                   ... 
FOKKER F28            1
ROBINSON R66          1
FAIRCHILD FH227       1
ANTONOV 124           1
GULFSTREAM G700       1
Name: AIRCRAFT, Length: 424, dtype: int64
4    36188
3     9422
1     3498
2     3485
5      376
Name: AC_MASS, dtype: int64


In [21]:
print(df['PHASE_OF_FLIGHT'].value_counts())
print(df['SPECIES'].value_counts())
print(df['NUM_STRUCK'].value_counts())

Approach        21748
Take-off Run    11858
Landing Roll    11559
Climb            7013
Descent           493
Taxi              143
Local             123
Parked             29
En Route            2
Unknown             1
Name: PHASE_OF_FLIGHT, dtype: int64
Unknown bird - small     17603
Unknown bird - medium     6119
Unknown bird              2637
Gulls                     2371
Mourning dove             1658
                         ...  
Chuck-will's-widow           1
Northern cardinal            1
Horse                        1
Grebes                       1
Purple gallinule             1
Name: SPECIES, Length: 533, dtype: int64
1                43670
2-10              8713
11-100             566
More than 100       20
Name: NUM_STRUCK, dtype: int64
